In [176]:
from collections import OrderedDict
from nltk.tokenize import word_tokenize
import numpy as np
from math import log2
from nltk import FreqDist
from numpy.linalg import svd
import matplotlib.pyplot as plt

In [158]:
document = ['This is the first document','This document is the second document','And this is the third one', 'Is this the first document']

tokenized_words = [word_tokenize(sent) for sent in document]
print("Tokenized words\n")
print(tokenized_words)

Tokenized words

[['This', 'is', 'the', 'first', 'document'], ['This', 'document', 'is', 'the', 'second', 'document'], ['And', 'this', 'is', 'the', 'third', 'one'], ['Is', 'this', 'the', 'first', 'document']]


In [159]:
lowered_words = [[w.lower() for w in word] for word in tokenized_words]
print("Lower cased words\n")
print(lowered_words)

Lower cased words

[['this', 'is', 'the', 'first', 'document'], ['this', 'document', 'is', 'the', 'second', 'document'], ['and', 'this', 'is', 'the', 'third', 'one'], ['is', 'this', 'the', 'first', 'document']]


In [160]:
tokens = []
for i in range(len(lowered_words)):
    tokens = tokens + lowered_words[i]
vocab = list(sorted(set(tokens)))
print(tokens)

['this', 'is', 'the', 'first', 'document', 'this', 'document', 'is', 'the', 'second', 'document', 'and', 'this', 'is', 'the', 'third', 'one', 'is', 'this', 'the', 'first', 'document']


In [161]:
occurrences = OrderedDict((name, OrderedDict((name, 0) for name in tokens)) for name in tokens)

for l in lowered_words:
    for i in range(len(l)):
        for item in l[:i] + l[i + 1:]:
            occurrences[l[i]][item] += 1

print(' ', ' '.join(occurrences.keys()))
co_occurances = []
for name, values in occurrences.items():
    co_occurances.append(values.values())
    #print(name, ' '.join(str(i) for i in values.values()))
    print(name,' '.join(str(i) for i in values.values()))

  this is the first document second and third one
this 0 4 4 2 4 1 1 1 1
is 4 0 4 2 4 1 1 1 1
the 4 4 0 2 4 1 1 1 1
first 2 2 2 0 2 0 0 0 0
document 4 4 4 2 2 2 0 0 0
second 1 1 1 0 2 0 0 0 0
and 1 1 1 0 0 0 0 1 1
third 1 1 1 0 0 0 1 0 1
one 1 1 1 0 0 0 1 1 0


In [162]:
co_words = list(occurrences.keys())

In [163]:
document

['This is the first document',
 'This document is the second document',
 'And this is the third one',
 'Is this the first document']

In [164]:
a = []
for name, values in occurrences.items():
    for i in range(9):
        a.append(values.popitem(9))

In [165]:
b = []
for i in range(len(a)):
    b.append(a[i][1])

In [166]:
b = np.reshape(b,(9,9))
print(b)

[[1 1 1 1 4 2 4 4 0]
 [1 1 1 1 4 2 4 0 4]
 [1 1 1 1 4 2 0 4 4]
 [0 0 0 0 2 0 2 2 2]
 [0 0 0 2 2 2 4 4 4]
 [0 0 0 0 2 0 1 1 1]
 [1 1 0 0 0 0 1 1 1]
 [1 0 1 0 0 0 1 1 1]
 [0 1 1 0 0 0 1 1 1]]


In [167]:
mat = np.zeros((9,9))
for i in range(8,-1,-1):
    for j in range(8,-1,-1):
        mat[i][j] = b[8-i][8-j]

In [168]:
mat

array([[1., 1., 1., 0., 0., 0., 1., 1., 0.],
       [1., 1., 1., 0., 0., 0., 1., 0., 1.],
       [1., 1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 1., 0., 2., 0., 0., 0., 0.],
       [4., 4., 4., 2., 2., 2., 0., 0., 0.],
       [2., 2., 2., 0., 2., 0., 0., 0., 0.],
       [4., 4., 0., 2., 4., 1., 1., 1., 1.],
       [4., 0., 4., 2., 4., 1., 1., 1., 1.],
       [0., 4., 4., 2., 4., 1., 1., 1., 1.]])

In [169]:
freqDist = FreqDist(tokens)

In [170]:
freqDist

FreqDist({'document': 4, 'this': 4, 'the': 4, 'is': 4, 'first': 2, 'third': 1, 'and': 1, 'second': 1, 'one': 1})

In [171]:
m,n = np.shape(mat)

pmi_mat = np.zeros((m,n))
for i in range(m):
    for j in range(n):
        if mat[i][j] == 0:
            pmi_mat[i][j] = 0
        else:
            w = co_words[i]
            c = co_words[j]
            #print(freqDist[w]*freqDist[c])
            pmi = log2((mat[i][j]*m)/(freqDist[w]*freqDist[c]))
            #print(int(mat[i][j]))
            if pmi <= 0:
                pmi_mat[i][j] = 0
            else:
                pmi_mat[i][j] = pmi

In [172]:
pmi_mat

array([[0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        1.169925, 1.169925, 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        1.169925, 0.      , 1.169925],
       [0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 1.169925, 1.169925],
       [0.169925, 0.169925, 0.169925, 0.      , 1.169925, 0.      ,
        0.      , 0.      , 0.      ],
       [1.169925, 1.169925, 1.169925, 1.169925, 0.169925, 2.169925,
        0.      , 0.      , 0.      ],
       [2.169925, 2.169925, 2.169925, 0.      , 2.169925, 0.      ,
        0.      , 0.      , 0.      ],
       [3.169925, 3.169925, 0.      , 3.169925, 3.169925, 3.169925,
        3.169925, 3.169925, 3.169925],
       [3.169925, 0.      , 3.169925, 3.169925, 3.169925, 3.169925,
        3.169925, 3.169925, 3.169925],
       [0.      , 3.169925, 3.169925, 3.169925, 3.169925, 3.169925,
        3.169925, 3.169925, 3.169925]])

In [173]:
u,s,v = svd(pmi_mat)
word_mat = np.matmul(u,np.diag(s))
print(word_mat)

[[-8.33217988e-01 -8.23451373e-01 -3.51932372e-16 -9.18241507e-16
  -5.10776822e-01 -7.98080016e-01  5.24929799e-01  4.28104856e-01
  -9.20592350e-02]
 [-8.33217988e-01 -8.23451373e-01  0.00000000e+00  3.21925857e-16
  -5.10776822e-01 -5.55625328e-02 -9.53622468e-01  4.28104856e-01
  -9.20592350e-02]
 [-8.33217988e-01 -8.23451373e-01  2.19957733e-16  2.38272510e-16
  -5.10776822e-01  8.53642549e-01  4.28692669e-01  4.28104856e-01
  -9.20592350e-02]
 [-5.75871494e-01  5.59169924e-01 -3.62930259e-16  6.71401448e-17
  -5.14333823e-01  5.19551069e-16 -6.62833513e-15 -7.06040948e-01
  -2.19125406e-01]
 [-2.21999942e+00  1.23269873e+00  1.31974640e-16 -4.91431914e-16
   1.91213334e+00 -3.89663302e-16  3.00365462e-15  3.05523230e-01
  -1.21676224e-01]
 [-2.55365096e+00  3.35190836e+00 -1.75966186e-16  8.78835807e-17
  -9.95524336e-01 -1.94831651e-16  2.79258700e-15  2.93889651e-01
   2.08639117e-02]
 [-8.57573524e+00 -3.71585671e-01 -1.29411646e+00 -2.24147546e+00
  -5.04360021e-03  3.2471941

In [174]:
s = np.diag(s)
word_mat = np.matmul(u,s[:,0:10])
print(word_mat)

[[-8.33217988e-01 -8.23451373e-01 -3.51932372e-16 -9.18241507e-16
  -5.10776822e-01 -7.98080016e-01  5.24929799e-01  4.28104856e-01
  -9.20592350e-02]
 [-8.33217988e-01 -8.23451373e-01  0.00000000e+00  3.21925857e-16
  -5.10776822e-01 -5.55625328e-02 -9.53622468e-01  4.28104856e-01
  -9.20592350e-02]
 [-8.33217988e-01 -8.23451373e-01  2.19957733e-16  2.38272510e-16
  -5.10776822e-01  8.53642549e-01  4.28692669e-01  4.28104856e-01
  -9.20592350e-02]
 [-5.75871494e-01  5.59169924e-01 -3.62930259e-16  6.71401448e-17
  -5.14333823e-01  5.19551069e-16 -6.62833513e-15 -7.06040948e-01
  -2.19125406e-01]
 [-2.21999942e+00  1.23269873e+00  1.31974640e-16 -4.91431914e-16
   1.91213334e+00 -3.89663302e-16  3.00365462e-15  3.05523230e-01
  -1.21676224e-01]
 [-2.55365096e+00  3.35190836e+00 -1.75966186e-16  8.78835807e-17
  -9.95524336e-01 -1.94831651e-16  2.79258700e-15  2.93889651e-01
   2.08639117e-02]
 [-8.57573524e+00 -3.71585671e-01 -1.29411646e+00 -2.24147546e+00
  -5.04360021e-03  3.2471941